In [2]:
import requests
import pandas as pd
from io import StringIO
import os
from dotenv import load_dotenv

In [6]:
load_dotenv()  # .env 파일 읽어오기
api_key = os.getenv('API_KEY')

def download_file(file_url, save_path):
    with open(save_path, 'wb') as f: # 저장할 파일을 바이너리 쓰기 모드로 열기
        response = requests.get(file_url) # 파일 URL에 GET 요청 보내기
        f.write(response.content) # 응답의 내용을 파일에 쓰기

# URL과 저장 경로 변수를 지정합니다.
url = f"https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php?tm1=19040401&obs=TA&stn=108&help=0&mode=0&authKey={api_key}"

save_file_path = 'data.csv'

# # 파일 다운로드 함수를 호출합니다.
download_file(url, save_file_path)

In [7]:
with open('data.csv', 'r', encoding='utf-8') as f:
    lines = f.readlines()

# 주석(#) 제거
data_lines = [line for line in lines if not line.startswith('#')]

# 데이터 프레임으로 변환
from io import StringIO

data_str = ''.join(data_lines)
df = pd.read_csv(StringIO(data_str), delim_whitespace=True, header=None)

# 기온 컬럼만 남기기 'TA AVG' = 10, 'TA MAX' = 11, 'TA MIN' = 13번째 열 (0부터 시작)
df_ta = df.iloc[:, [0, 10, 11, 13]]
df_ta.columns = ['날짜', 'TA_AVG', 'TA_MAX', 'TA_MIN']

df_ta['날짜'] = pd.to_datetime(df_ta['날짜'].astype(str))

df_ta.to_csv('TA_data.csv', index=False)

print(df_ta.head())

C:\Users\01028\AppData\Local\Temp\ipykernel_40768\3997389493.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(data_str), delim_whitespace=True, header=None)


          날짜  TA_AVG  TA_MAX  TA_MIN
0 1907-10-01    13.5    20.7     7.9
1 1907-10-02    16.2    22.0     7.9
2 1907-10-03    16.2    21.3    13.1
3 1907-10-04    16.5    22.0    11.2
4 1907-10-05    17.6    25.4    10.9


C:\Users\01028\AppData\Local\Temp\ipykernel_40768\3997389493.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ta['날짜'] = pd.to_datetime(df_ta['날짜'].astype(str))


In [8]:
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41787 entries, 0 to 41786
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      41787 non-null  datetime64[ns]
 1   TA_AVG  41787 non-null  float64       
 2   TA_MAX  41787 non-null  float64       
 3   TA_MIN  41787 non-null  float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 1.3 MB
